Speculator is not as fast as expected. I'll be profiling speculator against fsps in this notebook and try to find out whether there's some bottleneck.

In [1]:
import numpy as np 
# -- gqp_mc --
from gqp_mc import util as UT
from gqp_mc import fitters as Fitters
# -- plotting --
import matplotlib as mpl 
import matplotlib.pyplot as plt 
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False
%matplotlib inline

In [2]:
ifsps = Fitters.iFSPS()
ispeculate = Fitters.iSpeculator()

In [3]:
prior_fsps = ifsps._default_prior()
prior_speculator = ispeculate._default_prior()

In [4]:
bands_list = ispeculate._get_bands('desi')
print(bands_list)
# get filters
from speclite import filters as specFilter
filters = specFilter.load_filters(*tuple(bands_list))

['decam2014-g', 'decam2014-r', 'decam2014-z']


First FSPS evaluation takes extra long so lets get it out of the way 

In [5]:
theta_fsps = np.random.uniform(prior_fsps.min, prior_fsps.max)
ifsps.model(theta_fsps)

(array([[1.00100e+02, 1.03400e+02, 1.05600e+02, ..., 1.07987e+08,
         1.08988e+08, 1.10000e+08]]),
 array([[5.64535383e-49, 5.29076223e-49, 5.07261014e-49, ...,
         1.60669220e-08, 1.53556065e-08, 1.46854437e-08]]))

Now lets profile FSPS

In [6]:
%%timeit
theta_fsps = np.random.uniform(prior_fsps.min, prior_fsps.max)
ifsps.model(theta_fsps, zred=0.2)

The slowest run took 837.95 times longer than the fastest. This could mean that an intermediate result is being cached.
743 ms ± 1.8 s per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
theta_fsps = np.random.uniform(prior_fsps.min, prior_fsps.max)
ifsps.model_photo(theta_fsps, filters=filters, zred=0.2)

/anaconda2/envs/gqp/lib/python3.7/site-packages/speclite/filters.py:1466: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  values_no_units = values_no_units[values_slice]
/anaconda2/envs/gqp/lib/python3.7/site-packages/speclite/filters.py:1520: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  integrand = integrand[values_slice]


9.02 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now lets profile Speculator

In [8]:
%%timeit
theta_speculator = np.random.uniform(prior_speculator.min, prior_speculator.max)
ispeculate.model(theta_speculator, zred=0.2)

294 µs ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%%timeit
theta_speculator = np.random.uniform(prior_speculator.min, prior_speculator.max)
ispeculate.model_photo(theta_speculator, zred=0.2, filters=filters)

3.02 ms ± 39.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
theta_speculator = np.random.uniform(prior_speculator.min, prior_speculator.max)
ispeculate._emulator(theta_speculator)

/anaconda2/envs/gqp/lib/python3.7/site-packages/gqp_mc-0.1-py3.7.egg/gqp_mc/fitters.py:1388: RuntimeWarning: overflow encountered in exp
  layers.append((self._emu_beta[i] + (1.-self._emu_beta[i])*1./(1.+np.exp(-self._emu_alpha[i]*act[-1])))*act[-1])


202 µs ± 3.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
